In [1]:
train_docs_addr = '/kaggle/input/indian-doc-summary-dataset/IN-Abs/train-data/judgement'
train_sums_addr = '/kaggle/input/indian-doc-summary-dataset/IN-Abs/train-data/summary'

In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
import pandas as pd
import os
train_df = pd.DataFrame()

train_docs = []
train_sums = []
doc_num = []

for doc in os.listdir(train_docs_addr):
    with open(train_docs_addr + '/' + doc, "r") as file:
        train_docs.append(file.read())
        doc_num.append(doc)

for summ in os.listdir(train_sums_addr):
    with open(train_sums_addr + '/' + summ, "r") as file:
        train_sums.append(file.read())

In [4]:
train_df['Doc'] = doc_num
train_df['Text'] = train_docs
train_df['Summary'] = train_sums
train_df.head()

,Doc,Text,Summary
0,1893.txt,iminal Appeal No. 47 of 1963.\nAppeal by speci...,"The appellant published in his paper, which ha..."
1,1711.txt,minal Appeal No. 210 of 1963.\nAppeal by speci...,On the first information report lodged by the ...
2,4682.txt,Civil Appeal No. 2618 of 1983.\nFrom the Judgm...,The first respondent who lost to the appellant...
3,5450.txt,ivil Appeal No. 2991 of 1986.\nFrom the Judgme...,The appellant was sought to be evicted under s...
4,5064.txt,Civil Appeal No. 1733 of 1973.\nFrom the Judgm...,A firm (JTC) obtained the sole selling agency ...


In [5]:


# Open the file and read lines
with open("/kaggle/input/indian-doc-summary-dataset/IN-Abs/train-data/stats-IN-train.txt", "r") as file:
    lines = file.readlines()

# Extract the second column values
second_column_values = [int(line.split()[1]) for line in lines]

# Extract the second column values
third_column_values = [int(line.split()[2]) for line in lines]



In [6]:
import numpy as np
import statistics as stats
def list_metrics(lst):
    metrics = {
        "min": min(lst),
        "max": max(lst),
        "mean": np.mean(lst),
        "median": np.median(lst),
        "mode": stats.mode(lst),
        "std_dev": np.std(lst),
        "variance": np.var(lst),
        "sum": sum(lst),
        "count": len(lst),
        "range": max(lst) - min(lst),
        "25th_percentile": np.percentile(lst, 25),
        "75th_percentile": np.percentile(lst, 75),
        "iqr": np.percentile(lst, 75) - np.percentile(lst, 25),
    }
    return metrics

In [7]:
list_metrics(second_column_values)


{'min': 94,
 'max': 139867,
 'mean': 4371.1587482219065,
 'median': 3131.0,
 'mode': 2424,
 'std_dev': 5116.888605821558,
 'variance': 26182549.004386485,
 'sum': 30729246,
 'count': 7030,
 'range': 139773,
 '25th_percentile': 2019.0,
 '75th_percentile': 4974.75,
 'iqr': 2955.75}

In [8]:
list_metrics(third_column_values)

{'min': 0,
 'max': 27795,
 'mean': 841.2209103840682,
 'median': 640.0,
 'mode': 503,
 'std_dev': 900.0992865991495,
 'variance': 810178.7257362979,
 'sum': 5913783,
 'count': 7030,
 'range': 27795,
 '25th_percentile': 422.0,
 '75th_percentile': 965.0,
 'iqr': 543.0}

In [9]:
# train_df = train_df.sample(n=500, random_state=42).reset_index(drop=True)
train_df = train_df[:400]

In [10]:
train_df

,Doc,Text,Summary
0,1893.txt,iminal Appeal No. 47 of 1963.\nAppeal by speci...,"The appellant published in his paper, which ha..."
1,1711.txt,minal Appeal No. 210 of 1963.\nAppeal by speci...,On the first information report lodged by the ...
2,4682.txt,Civil Appeal No. 2618 of 1983.\nFrom the Judgm...,The first respondent who lost to the appellant...
3,5450.txt,ivil Appeal No. 2991 of 1986.\nFrom the Judgme...,The appellant was sought to be evicted under s...
4,5064.txt,Civil Appeal No. 1733 of 1973.\nFrom the Judgm...,A firm (JTC) obtained the sole selling agency ...
...,...,...,...
395,5.txt,Civil Appeal No. 8 of 1951.\nAppeal from the j...,S and B were sons of two brothers respectively...
396,1591.txt,"Appeals Nos. 652, 653 and 757 of 1962.\nAppeal...",The history of the Nathdwara Temple in the Dis...
397,3929.txt,N: Criminal Appeal No. 144 of 1972.\nAppeal by...,On a complaint by the Additional Registrar of ...
398,426.txt,iminal Appeal No. 86 of 1954.\nAppeal under Ar...,The High Court has no jurisdiction to grant ce...


In [11]:
from IPython.display import clear_output
!pip install trl
clear_output()

In [12]:
import torch
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import (
    AutoModelForSeq2SeqLM, 
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    get_linear_schedule_with_warmup
)
from peft import (
    LoraConfig, 
    TaskType, 
    get_peft_model,
    PeftModel,
    PeftConfig
)
from torch.utils.data import DataLoader
from torch.nn.functional import kl_div, log_softmax, softmax
from tqdm.auto import tqdm
import os
from typing import List, Dict, Any
import gc

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths and parameters
MODEL_PATH = '/kaggle/input/led-model/LED/IN_model'
OUTPUT_DIR = "./legal-led-lora-output"
MAX_INPUT_LENGTH = 4096  # LED model supports up to 16384 tokens
MAX_OUTPUT_LENGTH = 1024  # Adjust based on your summary lengths
BATCH_SIZE = 2  # Can increase with LoRA
GRADIENT_ACCUMULATION_STEPS = 2  # Reduced as memory is less of an issue
LEARNING_RATE = 2e-4  # Typically higher for LoRA
NUM_EPOCHS = 1
ENTAILMENT_WEIGHT = 0.7  # Weight for entailment in reward function
KL_WEIGHT = 0.3  # Weight for KL divergence in reward function
SAVE_STEPS = 500
EVAL_STEPS = 500

# LoRA specific parameters
LORA_RANK = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.1

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

def load_models_and_tokenizer():
    """Load model with LoRA and tokenizer."""
    print("Loading model and tokenizer with LoRA configuration...")
    
    try:
        # Load tokenizer first
        tokenizer = AutoTokenizer.from_pretrained('nsi319/legal-led-base-16384')
        
        # Load the main model with reduced precision to save memory
        base_model = AutoModelForSeq2SeqLM.from_pretrained(
            MODEL_PATH,
            torch_dtype=torch.float16,  # Use lower precision
        )
        base_model.gradient_checkpointing_enable()  # Add this line to save more memory
        base_model = base_model.to(device)  # Move to GPU manually
                
        # Configure LoRA
        peft_config = LoraConfig(
            task_type=TaskType.SEQ_2_SEQ_LM,
            inference_mode=False,
            r=LORA_RANK,
            lora_alpha=LORA_ALPHA,
            lora_dropout=LORA_DROPOUT,
            # Target specific attention layers for efficiency
            target_modules=["q_proj", "v_proj", "k_proj", "out_proj", "fc1", "fc2"],
        )
        
        # Create model with LoRA adapter
        model = get_peft_model(base_model, peft_config)
        
        # Print trainable parameters vs total parameters for verification
        model.print_trainable_parameters()
            
        print("Model and tokenizer loaded successfully")
        # Force garbage collection to free memory
        gc.collect()
        torch.cuda.empty_cache()
        
        return model, tokenizer
    
    except Exception as e:
        print(f"ERROR loading models: {e}")
        raise

def prepare_dataset(df, tokenizer):
    """Prepare and tokenize the dataset."""
    print(f"Preparing dataset with {len(df)} examples")
    
    try:
        # Convert DataFrame to Dataset
        dataset = Dataset.from_pandas(df)
        print(f"Dataset columns: {dataset.column_names}")
        
        # Define preprocessing function
        def preprocess_function(examples):
            inputs = examples["Text"]
            targets = examples["Summary"]
            
            # Tokenize inputs
            model_inputs = tokenizer(
                inputs, 
                max_length=MAX_INPUT_LENGTH,
                padding="max_length",
                truncation=True
            )
            
            # Tokenize targets
            with tokenizer.as_target_tokenizer():
                labels = tokenizer(
                    targets,
                    max_length=MAX_OUTPUT_LENGTH,
                    padding="max_length",
                    truncation=True
                )
            
            model_inputs["labels"] = labels["input_ids"]
            return model_inputs
        
        # Process the dataset
        print("Starting dataset tokenization...")
        tokenized_dataset = dataset.map(
            preprocess_function,
            batched=True,
            remove_columns=dataset.column_names,
            desc="Tokenizing dataset"
        )
        
        print(f"Dataset preparation complete. Feature names: {tokenized_dataset.column_names}")
        print(f"Dataset size: {len(tokenized_dataset)}")
        return tokenized_dataset
        
    except Exception as e:
        print(f"ERROR in dataset preparation: {e}")
        raise

def compute_entailment_score(input_ids, attention_mask, summary_ids, summary_attention_mask, model):
    """Calculate a proxy for entailment score between document and summary."""
    try:
        print(f"Computing entailment score. Input shape: {input_ids.shape}, Summary shape: {summary_ids.shape}")
        
        with torch.no_grad():
            # Get document embeddings - access LED encoder directly
            doc_outputs = model.base_model.led.encoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
            )
            doc_embedding = doc_outputs.last_hidden_state.mean(dim=1)
            
            # Get summary embeddings
            summary_outputs = model.base_model.led.encoder(
                input_ids=summary_ids,
                attention_mask=summary_attention_mask,
            )
            summary_embedding = summary_outputs.last_hidden_state.mean(dim=1)
            
            # Normalize embeddings
            doc_embedding = doc_embedding / (doc_embedding.norm(dim=1, keepdim=True) + 1e-8)
            summary_embedding = summary_embedding / (summary_embedding.norm(dim=1, keepdim=True) + 1e-8)
            
            # Calculate cosine similarity
            similarity = torch.bmm(
                doc_embedding.unsqueeze(1), 
                summary_embedding.unsqueeze(2)
            ).squeeze()
            
            # Transform to a probability-like score between 0 and 1
            entailment_score = (similarity + 1) / 2
            
            print(f"Entailment score: {entailment_score.item():.4f}")
            return entailment_score
            
    except Exception as e:
        print(f"ERROR in entailment calculation: {e}")
        return torch.tensor([0.5], device=device)  # Fallback value

def calculate_rewards(input_ids, attention_mask, generated_ids, generated_attention_mask, model, tokenizer):
    """Calculate rewards for a batch of generated summaries using only entailment scores."""
    try:
        print(f"Calculating rewards for batch of size {input_ids.shape[0]}")
        batch_size = input_ids.shape[0]
        rewards = torch.zeros(batch_size, device=device)
        
        # Initialize base reward value - higher than 0 to promote training stability
        base_reward = 0.3
        
        # We'll measure primarily with entailment
        for i in range(batch_size):
            # Calculate entailment score
            entailment_score = compute_entailment_score(
                input_ids[i:i+1], 
                attention_mask[i:i+1],
                generated_ids[i:i+1], 
                generated_attention_mask[i:i+1],
                model
            )
            
            # No KL divergence - replaced with simpler approach
            # Add length penalty to avoid extremely short summaries
            summary_length = (generated_ids[i] != tokenizer.pad_token_id).sum()
            length_factor = min(1.0, summary_length / 50)  # Scale up to 1.0 based on minimum desired length
            
            # Combine rewards - primarily entailment-based with length factor
            combined_reward = base_reward + ENTAILMENT_WEIGHT * entailment_score.item() * length_factor
            rewards[i] = combined_reward
            print(f"Reward for example {i+1}: {combined_reward:.4f}")
        
        print(f"Average batch reward: {rewards.mean().item():.4f}")
        return rewards
        
    except Exception as e:
        print(f"ERROR in reward calculation: {e}")
        # Return default rewards to avoid stopping training
        return torch.ones(input_ids.shape[0], device=device) * 0.5

def main(train_df):
    """Main function to run the training pipeline."""
    print("\n" + "="*50)
    print("STARTING LORA TRAINING PIPELINE")
    print("="*50)
    
    try:
        # Load model and tokenizer with LoRA
        model, tokenizer = load_models_and_tokenizer()
        
        # Prepare dataset
        tokenized_dataset = prepare_dataset(train_df, tokenizer)
        
        # Create data collator
        print("Creating data collator...")
        data_collator = DataCollatorForSeq2Seq(
            tokenizer=tokenizer,
            model=model,
            padding=True,
            return_tensors="pt"
        )
        
        # Create DataLoader
        print("Creating DataLoader...")
        train_dataloader = DataLoader(
            tokenized_dataset, 
            batch_size=BATCH_SIZE, 
            shuffle=True, 
            collate_fn=data_collator
        )
        print(f"Created DataLoader with {len(train_dataloader)} batches")
        
        # Initialize optimizer - using 8-bit Adam to save memory
        print("Initializing optimizer...")
        try:
            from bitsandbytes.optim import Adam8bit
            optimizer = Adam8bit(model.parameters(), lr=LEARNING_RATE)
            print("Using 8-bit Adam optimizer for memory efficiency")
        except ImportError:
            print("Bitsandbytes not available, using standard AdamW")
            optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
        
        # Create scheduler
        num_training_steps = len(train_dataloader) * NUM_EPOCHS
        num_warmup_steps = int(0.1 * num_training_steps)
        scheduler = get_linear_schedule_with_warmup(
            optimizer, 
            num_warmup_steps=num_warmup_steps, 
            num_training_steps=num_training_steps
        )
        
        # Training loop
        print("\n" + "="*50)
        print("STARTING TRAINING")
        print("="*50)
        
        model.train()
        
        global_step = 0
        total_loss = 0
        
        for epoch in range(NUM_EPOCHS):
            print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
            epoch_loss = 0
            epoch_rewards = []
            
            # Create progress bar for the epoch
            progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch+1}")
            
            for step, batch in progress_bar:
                print(f"\nProcessing batch {step+1}/{len(train_dataloader)} in epoch {epoch+1}")
                
                # Move batch to device
                batch = {k: v.to(device) for k, v in batch.items()}
                
                # Forward pass
                print("Running forward pass...")
                outputs = model(**batch)
                loss = outputs.loss
                print(f"Loss: {loss.item():.4f}")
                
                # Generate summaries for reward calculation
                print("Generating summaries for reward calculation...")
                with torch.no_grad():
                    generated_ids = model.generate(
                        input_ids=batch["input_ids"],
                        attention_mask=batch["attention_mask"],
                        max_length=MAX_OUTPUT_LENGTH,
                        num_beams=2,  # Reduced from 4 to save memory
                        early_stopping=True
                    )
                
                # Create attention mask for generated IDs
                generated_attention_mask = torch.ones_like(generated_ids)
                
                # Calculate rewards
                print("Calculating rewards...")
                rewards = calculate_rewards(
                    batch["input_ids"],
                    batch["attention_mask"],
                    generated_ids,
                    generated_attention_mask,
                    model,
                    tokenizer
                )
                
                # Calculate reward-weighted loss
                print("Calculating reward-weighted loss...")
                reward_weighted_loss = loss * rewards.mean()
                print(f"Reward-weighted loss: {reward_weighted_loss.item():.4f}")
                
                # Scale loss for gradient accumulation
                scaled_loss = reward_weighted_loss / GRADIENT_ACCUMULATION_STEPS
                scaled_loss.backward()
                
                # Update weights (with gradient accumulation)
                if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
                    print("Updating weights...")
                    # Gradient clipping to prevent exploding gradients
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    
                    optimizer.step()
                    scheduler.step()
                    optimizer.zero_grad()
                    
                    global_step += 1
                    
                    # Log metrics
                    avg_reward = rewards.mean().item()
                    epoch_rewards.append(avg_reward)
                    epoch_loss += loss.item()
                    
                    progress_bar.set_postfix({
                        "loss": f"{loss.item():.4f}",
                        "reward": f"{avg_reward:.4f}",
                        "lr": f"{scheduler.get_last_lr()[0]:.6f}"
                    })
                    
                    # Save checkpoint periodically
                    if global_step % SAVE_STEPS == 0:
                        checkpoint_dir = os.path.join(OUTPUT_DIR, f"checkpoint-{global_step}")
                        print(f"Saving LoRA checkpoint to {checkpoint_dir}")
                        model.save_pretrained(checkpoint_dir)
                        tokenizer.save_pretrained(checkpoint_dir)
                        
                        # Log example generation
                        if len(batch["input_ids"]) > 0:
                            input_text = tokenizer.decode(batch["input_ids"][0], skip_special_tokens=True)
                            generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
                            
                            print("\nExample generation:")
                            print(f"Input (truncated): {input_text[:200]}...")
                            print(f"Generated summary: {generated_text}")
                
                # Force garbage collection periodically
                if step % 10 == 0:
                    gc.collect()
                    torch.cuda.empty_cache()
            
            # Save model after each epoch
            epoch_dir = os.path.join(OUTPUT_DIR, f"epoch-{epoch+1}")
            print(f"Saving LoRA model after epoch {epoch+1} to {epoch_dir}")
            model.save_pretrained(epoch_dir)
            tokenizer.save_pretrained(epoch_dir)
            
            # Log epoch metrics
            avg_epoch_loss = epoch_loss / len(train_dataloader)
            avg_epoch_reward = sum(epoch_rewards) / len(epoch_rewards) if epoch_rewards else 0
            print(f"Epoch {epoch+1} metrics: Loss={avg_epoch_loss:.4f}, Reward={avg_epoch_reward:.4f}")
        
        print("\n" + "="*50)
        print("TRAINING COMPLETE")
        print("="*50)
        
        # Save final model
        final_model_dir = os.path.join(OUTPUT_DIR, "final-model")
        print(f"Saving final LoRA model to {final_model_dir}")
        model.save_pretrained(final_model_dir)
        tokenizer.save_pretrained(final_model_dir)
        
        return model, tokenizer
        
    except Exception as e:
        print("\n" + "="*50)
        print(f"CRITICAL ERROR IN TRAINING: {e}")
        print("="*50)
        import traceback
        traceback.print_exc()
        
        # Try to save the model if it exists
        try:
            if 'model' in locals():
                emergency_save_dir = os.path.join(OUTPUT_DIR, "emergency-save")
                print(f"Attempting emergency save to {emergency_save_dir}")
                model.save_pretrained(emergency_save_dir)
                if 'tokenizer' in locals():
                    tokenizer.save_pretrained(emergency_save_dir)
                print("Emergency save successful")
        except Exception as save_error:
            print(f"Emergency save failed: {save_error}")
        
        raise

def load_and_merge_lora_model(base_model_path, lora_model_path):
    """Load a LoRA model and merge it with the base model for inference."""
    print(f"Loading and merging LoRA model from {lora_model_path}")
    
    try:
        # Load the base model
        base_model = AutoModelForSeq2SeqLM.from_pretrained(
            base_model_path,
            torch_dtype=torch.float16
        )
        base_model = base_model.to(device)  # Move to GPU manually
        
        # Load the LoRA configuration
        peft_config = PeftConfig.from_pretrained(lora_model_path)
        
        # Load the LoRA model
        model = PeftModel.from_pretrained(base_model, lora_model_path)
        
        # Merge the models (optional, for deployment efficiency)
        # model = model.merge_and_unload()
        
        return model
    
    except Exception as e:
        print(f"ERROR loading and merging model: {e}")
        raise

def test_model(model, tokenizer, test_df, num_examples=3):
    """Test the model on sample documents."""
    print("\n" + "="*50)
    print(f"TESTING MODEL ON {num_examples} EXAMPLES")
    print("="*50)
    
    model.eval()
    
    try:
        for i in range(min(num_examples, len(test_df))):
            # Get a sample document
            sample_doc = test_df.iloc[i]["Text"]
            original_summary = test_df.iloc[i]["Summary"]
            
            print(f"\nProcessing test example {i+1}/{num_examples}")
            
            # Tokenize
            print("Tokenizing input...")
            inputs = tokenizer(
                sample_doc,
                max_length=MAX_INPUT_LENGTH,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            ).to(device)
            
            # Generate summary
            print("Generating summary...")
            with torch.no_grad():
                generated_ids = model.generate(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    max_length=MAX_OUTPUT_LENGTH,
                    num_beams=4,
                    early_stopping=True
                )
            
            # Decode
            generated_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            
            print(f"\nExample {i+1}:")
            print(f"Original document (truncated): {sample_doc[:200]}...")
            print(f"Original summary: {original_summary}")
            print(f"Generated summary: {generated_summary}")
            
    except Exception as e:
        print(f"ERROR during testing: {e}")
        import traceback
        traceback.print_exc()

# Example usage of the code
if __name__ == "__main__":
    print("\n" + "="*50)
    print("STARTING SCRIPT")
    print("="*50)
    
    try:
        # Verify train_df exists and has the right structure
        print("Checking training data...")
        print(f"train_df type: {type(train_df)}")
        print(f"train_df shape: {train_df.shape}")
        print(f"train_df columns: {train_df.columns.tolist()}")
        
        if "Text" not in train_df.columns or "Summary" not in train_df.columns:
            print("ERROR: train_df missing required columns 'Text' and 'Summary'")
            raise ValueError("Dataset missing required columns")
            
        # Train the model
        model, tokenizer = main(train_df)
        
        # Test the model
        test_model(model, tokenizer, train_df)
        
    except Exception as e:
        print("\n" + "="*50)
        print(f"SCRIPT FAILED: {e}")
        print("="*50)
        import traceback
        traceback.print_exc()

Using device: cuda

STARTING SCRIPT
Checking training data...
train_df type: <class 'pandas.core.frame.DataFrame'>
train_df shape: (400, 3)
train_df columns: ['Doc', 'Text', 'Summary']

STARTING LORA TRAINING PIPELINE
Loading model and tokenizer with LoRA configuration...


tokenizer_config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Calculating rewards...
Calculating rewards for batch of size 2
Computing entailment score. Input shape: torch.Size([1, 4096]), Summary shape: torch.Size([1, 809])
Entailment score: 0.8535
Reward for example 1: 0.8975
Computing entailment score. Input shape: torch.Size([1, 4096]), Summary shape: torch.Size([1, 809])
Entailment score: 0.7593
Reward for example 2: 0.8315
Average batch reward: 0.8645
Calculating reward-weighted loss...
Reward-weighted loss: 4.4102

Processing batch 194/200 in epoch 1
Running forward pass...
Loss: 2.2305
Generating summaries for reward calculation...
Calculating rewards...
Calculating rewards for batch of size 2
Computing entailment score. Input shape: torch.Size([1, 4096]), Summary shape: torch.Size([1, 1024])
Entailment score: 0.9331
Reward for example 1: 0.9532
Computing entailment score. Input shape: torch.Size([1, 4096]), Summary shape: torch.Size([1, 1024])
Entailment score: 0.9404
Reward for example 2: 0.9583
Average batch reward: 0.9557
Calculating 

Input ids are automatically padded from 968 to 1024 to be a multiple of `config.attention_window`: 1024


Calculating rewards...
Calculating rewards for batch of size 2
Computing entailment score. Input shape: torch.Size([1, 4096]), Summary shape: torch.Size([1, 968])
Entailment score: 0.8389
Reward for example 1: 0.8872
Computing entailment score. Input shape: torch.Size([1, 4096]), Summary shape: torch.Size([1, 968])
Entailment score: 0.9180
Reward for example 2: 0.9426
Average batch reward: 0.9149
Calculating reward-weighted loss...
Reward-weighted loss: 2.9716

Processing batch 198/200 in epoch 1
Running forward pass...
Loss: 2.0586
Generating summaries for reward calculation...
Calculating rewards...
Calculating rewards for batch of size 2
Computing entailment score. Input shape: torch.Size([1, 4096]), Summary shape: torch.Size([1, 1024])
Entailment score: 0.8975
Reward for example 1: 0.9282
Computing entailment score. Input shape: torch.Size([1, 4096]), Summary shape: torch.Size([1, 1024])
Entailment score: 0.7637
Reward for example 2: 0.8346
Average batch reward: 0.8814
Calculating 

Input ids are automatically padded from 803 to 1024 to be a multiple of `config.attention_window`: 1024


Calculating rewards...
Calculating rewards for batch of size 2
Computing entailment score. Input shape: torch.Size([1, 4096]), Summary shape: torch.Size([1, 803])
Entailment score: 0.8682
Reward for example 1: 0.9077
Computing entailment score. Input shape: torch.Size([1, 4096]), Summary shape: torch.Size([1, 803])
Entailment score: 0.8418
Reward for example 2: 0.8893
Average batch reward: 0.8985
Calculating reward-weighted loss...
Reward-weighted loss: 1.9251
Updating weights...
Saving LoRA model after epoch 1 to ./legal-led-lora-output/epoch-1
Epoch 1 metrics: Loss=2.0246, Reward=0.8937

TRAINING COMPLETE
Saving final LoRA model to ./legal-led-lora-output/final-model

TESTING MODEL ON 3 EXAMPLES

Processing test example 1/3
Tokenizing input...
Generating summary...

Example 1:
Original document (truncated): iminal Appeal No. 47 of 1963.
Appeal by special leave from the judgment and order, dated January 29, 1963 of the Allahabad High Court in Criminal Appeal No. 998 of 1962.
M. K. Ram